# Импорт библиотек

In [ ]:
! pip install keybert
! pip install fuzzy-c-means
! pip install transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 237 kB 7.0 MB/s 
     |████████████████████████████████| 85 kB 5.1 MB/s 
     |████████████████████████████████| 51 kB 7.7 MB/s 
     |████████████████████████████████| 4.9 MB 60.0 MB/s 
     |████████████████████████████████| 1.3 MB 55.4 MB/s 
     |████████████████████████████████| 163 kB 49.0 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=75c4af37b8020cd06747d251c0cd805b673b7718ebd23a4eda0b39eee58db59a
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: typer
    Found existing installation: typer 0.4

In [ ]:
import numpy as np
import pandas as pd
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from sklearn.cluster import AgglomerativeClustering
import csv
from transformers import AutoTokenizer, AutoModel
from scipy import spatial
from sklearn.neighbors import NearestNeighbors
from datetime import date
import dateutil.relativedelta
from scipy import spatial

# Работа с данными

In [ ]:
rbk_news = pd.read_csv('/content/rbk_news.csv')
forbes_news = pd.read_csv('/content/forbes_news.csv')
gazeta_news = pd.read_csv('/content/gazeta_news.csv', engine='python')
kommersant_news = pd.read_csv('/content/kommersant_news.csv', engine='python')

In [ ]:
# data cleaning
rbk_news['title']= rbk_news['title'].apply(lambda x: x.strip())

In [ ]:
# collect all news to one dataset
df = rbk_news[['content', 'date', 'title', 'url']].append(forbes_news[['content', 'date', 'title', 'url']].
                                                   append(gazeta_news[['content', 'date', 'title', 'url']]).
                                                   append(kommersant_news[['content', 'date', 'title', 'url']]))

In [ ]:
len(gazeta_news)

999

In [ ]:
df = df.dropna()
df = df.astype({'content': pd.StringDtype(),'date': np.Datetime64, 'title': pd.StringDtype(), 'url': pd.StringDtype()})
df = df.sort_values(by='date')

In [ ]:
full_data_df = df[df['date'] > '2021-05-04']
# sort news by date
full_data_df = full_data_df.sort_values(by='date')

In [ ]:
full_data_df = full_data_df.set_axis(range(len(full_data_df)), axis='index')
full_data_df.to_csv('full_news.csv')

In [ ]:
full_data_df.head()

,content,date,title,url
0,Премьер-министр Михаил Мишустин утвердил списо...,2021-05-05,Мишустин дал поручения по реализации послания ...,https://www.kommersant.ru/doc/4801277
1,Польша ждет наплыва украинцев во время сбора у...,2021-05-05,Украинцы рассказали о рабском труде на заработ...,https://www.gazeta.ru/business/news/2021/05/05...
2,Генеральный директор Всемирной торговой органи...,2021-05-05,ВТО призвала увеличить поставки вакцин от COVI...,https://www.gazeta.ru/business/news/2021/05/05...
3,На строительство газопровода «Северный поток —...,2021-05-05,В ФРГ выступили за мораторий на «Северный пото...,https://www.gazeta.ru/business/news/2021/05/05...
4,"Индекс Московской биржи, являющийся одним из о...",2021-05-05,Индекс Московской биржи обновил исторический м...,https://www.gazeta.ru/business/news/2021/05/05...


In [ ]:
full_data_df.tail()

,content,date,title,url
24550,Ведущий аналитик отдела глобальных исследовани...,2022-10-09,Аналитик назвал условия для ослабления рубля,https://www.gazeta.ru/business/news/2022/10/09...
24551,Доцент департамента публичного права факультет...,2022-10-09,Россиянам рассказали о возможности увеличить п...,https://www.gazeta.ru/business/news/2022/10/09...
24552,Обозреватель The Washington Post Эмили Раухала...,2022-10-09,WP: Норвегия «неприлично обогащается» на украи...,https://www.gazeta.ru/business/news/2022/10/09...
24553,Аналитик ФГ «Финам» Алексей Калачев в разговор...,2022-10-09,Аналитик оценил влияние Лондонской биржи метал...,https://www.gazeta.ru/business/news/2022/10/09...
24554,"Президент Владимир Путин, поздравляя работнико...",2022-10-09,"Путин заявил, что Россия готова помогать в реш...",https://www.gazeta.ru/business/news/2022/10/09...


In [ ]:
# get news from 1 month ago
today = date.today()
month_ago = np.datetime64(today - dateutil.relativedelta.relativedelta(months=1))
recent_news = full_data_df[full_data_df['date'] > month_ago]
recent_news = recent_news.set_axis(range(len(recent_news)), axis='index')
recent_news.to_csv('recent_news.csv')

In [ ]:
len(recent_news)

2460

# дайджесты

In [ ]:
class Helper:
    def __init__(self, value, index, count=0):
        self.value = value
        self.index = index
        self.count = count
class Pipeline():
    def __init__(self, df, kwd_model, 
                 emb_model,
                 role,  
                 parsed = False):
        self.df = df  # dataframe of news with columns: ['title', 'content', 'date', 'url']
        self.kwd_model = kwd_model  # keyword extraction model
        self.emb_model = emb_model  # embedding model
        self.parsed = parsed  # boolean, if True, extract values from local csv file
        self.role = role  # str, either 'buh' or 'businessman'
        self.role_words = []  # list of words similar to what news we want to recieve
        self.three_gramms = []  # list of keywords for all news
        self.keyword_embs = []  # list of keywords embeddings for all news
        self.emb_prof = 0  # embedding for list of role_words
    
    # sets role words depending on role 
    def _set_role_words(self):
        if self.role == 'buh':
          self.role_words =  ['бухгалтер', 'бухгалтерия', 'бухгалтерские проводки', 
            'корреспонденция счетов', 'учетная политика организации', 'НК РФ', 
            'Минфин России', 'министерство финансов', 'фонд социального страхования', 
            'минэкономразвития', 'форма документа', 'учет', 'НДФЛ', 'налог', 
            'взнос', 'сделка', '1c', '1с', 'платеж', 'дебет', 'квартальный отчет', 
            'закон', 'квартал', 'расчет зарплаты', 'отчетность',
            'ведомость', 'индексация', 'касса', 'платежка', 'бухучет', 
            'бухгалтерский учет', 'приход', 'оборот', 'договор','декларация', 'бюджет']

        elif self.role == 'businessman':
            self.role_words = ['увеличить прибыль', 'индивидуальный предприниматель',
                  'влияние на предпринимателей', 'поддержка бизнеса', 
                  'венчурный инвестор', 'стартап', 'инвестиции', 
                  'конкуренция бизнеса', 'субсидии от государства', 
                  'кадастровая стоимость', 'задачи малого бизнеса',
                  'льгота бизнесу', 'кредитные каникулы', 'самозанятый',
                  'общество с ограниченной ответственностью', 'акционерное общество']
    
    # extracts keywords from news
    def _get_3_gramms(self):
        if self.parsed == False:
          with open('keywords.csv', 'w', newline='') as csvfile:
            fieldnames = ['title', 'keywords']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for idx, text in tqdm(enumerate(self.df['content'])):
                keywords = self.kwd_model.extract_keywords(text, 
                                        keyphrase_ngram_range=(3, 3), top_n=5)  
                for keyword in keywords: #iterate through list of tuples. Keyword is tuple (3 words, score)
                    self.three_gramms.append(keyword[0])
                keywords = [keyword[0] for keyword in keywords]
                writer.writerow({'title': self.df['title'][idx], 'keywords': keywords})
            print('3-gramms', self.three_gramms)
        else:
          keywords_df = pd.read_csv('/content/keywords.csv')
          keywords_df['keywords'] = keywords_df['keywords'].apply(
              lambda keyword_list: keyword_list.replace("'", '').replace(
                  '[', '').replace(']', '').split(', '))
          for i in range(len(keywords_df)):
            for j in range(5):
              self.three_gramms.append(keywords_df['keywords'][i][j])
    
    # finds embeddings for each of the keywords
    def _get_embeddings(self):
        with open('keywords_emb.csv', 'w', newline='') as csvfile:
            fieldnames = ['embedding']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            for keyword in self.three_gramms:
              self.keyword_embs.append(np.array(self.emb_model.encode(keyword)))
              writer.writerow({'embedding': np.array(self.emb_model.encode(keyword))})

    # gets embedding for role words:
    # finds embeddings for role words and averages them
    def _get_embedding_prof(self):
        prof_embs = []
        for word in self.role_words:
          prof_embs.append(np.array(self.emb_model.encode(word)))
        for emb in prof_embs:
          self.emb_prof += emb
        self.emb_prof /= len(self.role_words)

        with open('prof_emb.csv', 'w', newline='') as csvfile:
          fieldnames = ['embedding']
          writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
          writer.writeheader()
          writer.writerow({'embedding': np.array(self.emb_prof)})

    # sorts array of indexes
    def sortByFrequencyAndIndex(self, arr): 
        if arr is None or len(arr) < 2:
            return
        hm = {}
        for i in range(len(arr)):
            hm.setdefault(arr[i], Helper(arr[i], i)).count += 1
        values = [*hm.values()]
    
        values.sort(key=lambda x: (-x.count, x.index))
    
        k = 0
        for data in values:
            for j in range(data.count):
                arr[k] = data.value
                k += 1

    # returns top n related to role news
    def _get_n_closest_news_to_role(self, n):
        # get distance for each of the keywords with role words embedding
        distances = [] 
        for keyword_emb in tqdm(self.keyword_embs):
            cos_sim = 1 - spatial.distance.cosine(keyword_emb, self.emb_prof)
            distances.append(cos_sim)
        distances = np.array(distances)
        # get indicies of keywords that are nearest to role words embedding
        sorted_indicies = np.argsort(distances)[:n] 
        # get most related news 
        top_news_indices = []
        for idx in sorted_indicies:
            news_idx = idx // 5
            top_news_indices.append(news_idx)
        self.sortByFrequencyAndIndex(np.array(top_news_indices))
        top_news_indices = pd.unique(top_news_indices)
        with open('best_news.csv', 'w', newline='') as csvfile:
          fieldnames = ['title', 'content', 'url', 'news_idx']
          writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
          writer.writeheader()
          for news_idx in top_news_indices:
            writer.writerow({'title': self.df['title'][news_idx], 
                             'content': self.df['content'][news_idx],
                             'news_idx': news_idx, 
                             'url': self.df['url'][news_idx]})    

In [ ]:
keyword_model = KeyBERT(model='DeepPavlov/rubert-base-cased')
sentence_model = SentenceTransformer('cointegrated/rubert-tiny')
nbrs = NearestNeighbors(n_neighbors=40, algorithm='ball_tree')

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/DeepPavlov_rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at /root/.cache/torch/sentence_tra

In [ ]:
pp = Pipeline(recent_news, kwd_model=keyword_model, emb_model=sentence_model, role = 'buh')
pp._set_role_words()
pp._get_3_gramms()
pp._get_embeddings()
pp._get_embedding_prof()
pp._get_n_closest_news_to_role(10)

781it [1:18:27, 17.99s/it]

In [ ]:
best = pd.read_csv('/content/best_news.csv')
for t in best['title']:
  print(t)

FT: ЕС окажет помощь пострадавшим от антироссийских санкций компаниям 
Аналитик Мехтиев назвал обесценивание одним из основных рисков при снятии наличных со счета 
Гутерриш встретится с Байденом для обсуждения цен на энергоносители 
Россияне стали покупать системы видеонаблюдения почти вдвое чаще за восемь последних месяцев 
Транзит контейнеров по железным дорогам России упадет на 25%
Переориентация на внутренний рынок: новые реалии мебельной отрасли России
Bloomberg: европейцы сорвали план ЕС по энергосбережению из-за отказа экономить газ 
Банки зафиксировали увеличение спроса россиян на вложения в драгметаллы 


In [ ]:
pp = Pipeline(recent_news, kwd_model=keyword_model, emb_model=sentence_model, role = 'businessman')
pp._set_role_words()
pp._get_3_gramms()
pp._get_embeddings()
pp._get_embedding_prof()
pp._get_n_closest_news_to_role(10)

2460it [16:39,  2.46it/s]


3-gramms ['импорт российской нефти', 'цены российской нефти', 'нефти декабря нефтепродуктов', 'закупленной нефти ранее', 'цену закупленной нефти', 'отсутствия газа трубопровода', 'ускорится до расходы', 'энергоресурсов приведут сокращению', 'за отсутствия газа', 'приведет сокращению производств', 'россии дмитрий биричевский', 'сотрудничеству хабаровским краем', 'дмитрий биричевский заявлял', 'интерес сотрудничеству хабаровским', 'хабаровским краем проявляют', 'электроэнергии евросоюза добившись', 'еврокомиссия намерена предложить', 'заявил что евросоюз', 'что еврокомиссия обязала', 'что еврокомиссия намерена', 'усугубить энергокризис евросоюзе', 'мировой рынок скептически', 'усугубить энергетический кризис', 'цены на российскую', 'энергокризис евросоюзе ес', 'газа нафтогаз украины', 'нафтогаз украины анонсировала', 'природного газа нафтогаз', 'транспортировке переработке нефти', 'на закупку газа', 'сторонники молдавской оппозиционной', 'президента кишиневе объявили', 'оппозиционной пар

100%|██████████| 12295/12295 [00:00<00:00, 13199.03it/s]


In [ ]:
keywords = pd.read_csv('/content/keywords.csv')
keywords[:10]

,title,keywords
0,Минфин США: запрет на морские перевозки россий...,"['импорт российской нефти', 'цены российской н..."
1,Bloomberg: производство в Германии упадет на 6...,"['отсутствия газа трубопровода', 'ускорится до..."
2,Дегтярев: провинциям КНР предложили перевозить...,"['россии дмитрий биричевский', 'сотрудничеству..."
3,Министр Кордейру: Португалия не согласна с ЕС ...,"['электроэнергии евросоюза добившись', 'евроко..."
4,Аналитик Citigroup Морс допустил ухудшение эне...,"['усугубить энергокризис евросоюзе', 'мировой ..."
5,"Украина запросила $1,5 млрд у Экспортно-импорт...","['газа нафтогаз украины', 'нафтогаз украины ан..."
6,В Италии растет недовольство ценами на электро...,"['сторонники молдавской оппозиционной', 'прези..."
7,Пищепром Украины готовится к повышению цен на ...,"['рублей производители сахара', 'пищевая промы..."
8,Evrensel: Европе грозит холодная зима из-за га...,"['ущерб понесут европейцы', 'введут потолок це..."
9,В Госдепе США прокомментировали заявление Росс...,"['продовольствием украины российский', 'его сл..."


In [ ]:
best = pd.read_csv('/content/best_news.csv')
for t in best['title']:
  print(t)

Аналитик Мехтиев назвал обесценивание одним из основных рисков при снятии наличных со счета 
Путин призвал производить конкурентоспособные двигатели для гражданской авиации 
Силуанов назвал предстоящий бюджет России самым сложным в его карьере 
Вице-премьер Молдавии сообщил о сокращении «Газпромом» поставок газа на треть в октябре 
Вице-премьер Молдавии Спыну заявил о сокращении потребления газа в стране более чем на 30% 
Процедуру банкротства российской «дочки» Google приостановили из-за неуплаты штрафа 
Президент Путин заявил, что экономический «блицкриг» ряда стран против России не сработал 
Министр Польши Москва заявила, что предприятиям порекомендуют уменьшить потребление энергии 
Международные резервы России сократились за неделю на $4,5 млрд или на 0,8% 
